## Integrating 2D functions by avoiding singularity ## 

**The code below is partly from scipy.quad library and I have used 3-4 _Stack Overflow_ responses. I have lost the _Stack Overflow_ sources, so I am not able to cite them. All sources that I used were license-free for modifications.** 

_I will still not call these as my codes._

The codes below integrates 2D functions by avoiding the the singularity area (poles area). Codes provided in Python library `scipy.integrate` `dblquad` gives an `Exception` error when singularity are encountered. 
The avoidance of singularity area contributes to error in integral computation. This can be minimized by using higher level of precision, i.e., reducing the area where singularity exists. The information on the location of singularity points are not a prerequisite of the developed code.

The execution of code can be slow for 2D functions. Low level precision <10 should be attempted as trial attempts before targeting a more precise results.

The main code is followed by an example, which can be modified. 

**The required libraries**

In [96]:
import numpy as np
from tqdm import tqdm

**The modified exception error class.**

In [89]:
class Int_Error(Exception):
    
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return self.message
        return "Error Message"

### The main code ###

We first define the **_Int_pole_** `class` and then define the `function` **_int1D_**.


In [90]:
class Integrate:
    
    def __init__(self, function):
        self.function = function
        self.error = 0
        self.sign = 1
        
    def int2D(self, limit_list, precision):
        
        x_list, y_list = limit_list
        (a, b), (c, d) = x_list, y_list
        x_points, y_points = (b - a) * precision, (d - c) * precision
        xs, ys = np.linspace(a, b, int(x_points)), np.linspace(c, d, int(y_points))
        
        integral = 0
        sub_sum = 0
        sup_sum = 0
        
        for i in tqdm(range(len(xs) - 1)):
            delta_x = xs[i + 1] - xs[i]
            for j in range(len(ys) - 1):
                delta_y = ys[j + 1] - ys[j]
                delta = delta_x * delta_y
                try:
                    f1 = self.function(xs[i], ys[j])
                    sub_area = f1 * delta
                    f2 = self.function(xs[i + 1], ys[j + 1])
                    sup_area = f2 * delta

                    area = (f2 + f1) / 2 * delta
                    integral += area
                    sub_sum += sub_area
                    sup_sum += sup_area
                    
                except ZeroDivisionError:
                    print(f"\nAvoided pole\n")

        self.error = sup_sum - sub_sum
        return integral

**Let us check the code using the following test function which has a pole at $x=0$:**

$$
\sin(x^2+y^2)/x
$$

**_Note_**: Make the precision value low or long time will be required. May be less than **10** to start with.

In [91]:
def test_fun(x, y):
    return np.sin(x ** 2 + y ** 2)/x

In [97]:
integral = Integrate(test_fun)
result = integral.int2D([[-100, 100], [-100, 100]], precision=3)

print("The result is", result)
print("\nThe accuracy of this result is", integral.error)

100%|██████████| 599/599 [00:02<00:00, 257.84it/s]

The result is 3.7874583569234843e-13

The accuracy of this result is -0.00909291495192107


**We check the 2D function here using Standard Python library `scipy.quad` function `dblquad`.
Check the [link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.dblquad.html#scipy.integrate.dblquad) for details of scipy.quad dblquad function.**

In the example we integrate using `dblquad` the same test function as we did above with our code.

In [98]:
from scipy import integrate
f = lambda y, x: np.sin(x ** 2 + y ** 2)/x
integrate.dblquad(f, -100, 100, lambda x: -100, lambda x: 100)

<ipython-input-98-770403a857cb>:2: RuntimeWarning: invalid value encountered in double_scalars
  f = lambda y, x: np.sin(x ** 2 + y ** 2)/x
<ipython-input-98-770403a857cb>:2: RuntimeWarning: divide by zero encountered in double_scalars
  f = lambda y, x: np.sin(x ** 2 + y ** 2)/x


(nan, 30129.507465093688)